# Cleaner after all projects combined

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# Show all columns
pd.set_option('display.max_columns', None)


Read JSON file

In [2]:
df = pd.read_json('../data/kickstarter.json', lines = True)
df.shape

(429463, 129)

Check data head

In [3]:
df.head()

,id,photo_key,photo_full,photo_ed,photo_med,photo_little,photo_small,photo_thumb,photo_1024x576,photo_1536x864,name,blurb,goal,pledged,state,slug,disable_communication,country,country_displayable_name,currency,currency_symbol,currency_trailing_code,deadline,state_changed_at,created_at,launched_at,staff_pick,is_starrable,backers_count,static_usd_rate,usd_pledged,converted_pledged_amount,fx_rate,usd_exchange_rate,current_currency,usd_type,creator_id,creator_name,creator_slug,creator_is_registered,creator_is_email_verified,creator_chosen_currency,creator_is_superbacker,creator_avatar_thumb,creator_avatar_small,creator_avatar_medium,creator_urls_web_user,creator_urls_api_user,location_id,location_name,location_slug,location_short_name,location_displayable_name,location_localized_name,location_country,location_state,location_type,location_is_root,location_expanded_country,location_urls_web_discover,location_urls_web_location,location_urls_api_nearby_projects,category_id,category_name,category_analytics_name,category_slug,category_position,category_parent_id,category_parent_name,category_color,category_urls_web_discover,video_id,video_status,video_hls,video_hls_type,video_high,video_high_type,video_base,video_base_type,video_tracks,video_width,video_height,video_frame,profile_id,profile_project_id,profile_state,profile_state_changed_at,profile_name,profile_blurb,profile_background_color,profile_text_color,profile_link_background_color,profile_link_text_color,profile_link_text,profile_link_url,profile_show_feature_image,profile_background_image_opacity,profile_should_show_feature_image_section,profile_feature_image_attributes_image_urls_default,profile_feature_image_attributes_image_urls_baseball_card,spotlight,urls_web_project,urls_web_rewards,percent_funded,is_liked,is_disliked,is_launched,prelaunch_activated,source_url,profile_feature_image_attributes_id,video,profile_background_image_attributes_id,profile_background_image_attributes_image_urls_default,profile_background_image_attributes_image_urls_baseball_card,is_in_post_campaign_pledging_phase,creator_has_admin_message_badge,creator_is_ksr_admin,creator_ppoHasAction,creator_ppo_has_action,friends,is_starred,is_backing,permissions,urls_api_star,urls_api_message_creator,urls_web_message_creator,photo_1024x768,photo_1536x1152,profile_should_show_feature_image
0,1293,assets/042/569/685/ff63ea0eb84ce1e987646f97a06...,https://i.kickstarter.com/assets/042/569/685/f...,https://i.kickstarter.com/assets/042/569/685/f...,https://i.kickstarter.com/assets/042/569/685/f...,https://i.kickstarter.com/assets/042/569/685/f...,https://i.kickstarter.com/assets/042/569/685/f...,https://i.kickstarter.com/assets/042/569/685/f...,https://i.kickstarter.com/assets/042/569/685/f...,https://i.kickstarter.com/assets/042/569/685/f...,Animal Instincts Poker Playing Cards & Tarot I...,Fantasy playing card decks combine the structu...,10000.0,36513.00,successful,animal-instincts-oracle-and-playing-card-decks,False,US,the United States,USD,$,True,1697119820,2023-10-12 14:10:20,2023-04-29 18:48:15,2023-09-12 14:10:20,0.0,0.0,491,1.000000,36513.000000,36513.0,1.000000,1.000000,USD,international,1276914410,David Bollt,minds-eye,NaN,NaN,None,NaN,https://i.kickstarter.com/assets/030/121/212/1...,https://i.kickstarter.com/assets/030/121/212/1...,https://i.kickstarter.com/assets/030/121/212/1...,https://www.kickstarter.com/profile/minds-eye,https://api.kickstarter.com/v1/users/127691441...,2356566.0,Asheville,asheville-nc,"Asheville, NC","Asheville, NC",Asheville,US,NC,Town,0.0,United States,https://www.kickstarter.com/discover/places/as...,https://www.kickstarter.com/locations/ashevill...,https://api.kickstarter.com/v1/discover?signat...,273,Playing Cards,Playing Cards,games/playing cards,4,12.0,Games,51627,http://www.kickstarter.com/discover/categories...,1241920.0,successful,https://v2.kickstarter.com/1713133893-L%2B7DXD...,application/x-mpegURL,https://v2.kickstarter.com/1713133893-L%2B7DXD...,"video/mp4; codecs="

# Clean column types

## Change all dates to datetime

In [4]:
date_columns = ['profile_state_changed_at', 'created_at', 'launched_at', 'state_changed_at']
df[date_columns] = df[date_columns].apply(pd.to_datetime, unit='s')

## Add USD pledged and USD goal

In [5]:
df['usd_pledged'] = df['pledged'] * df['static_usd_rate']
df['usd_goal'] = df['goal'] * df['static_usd_rate']

## Reorder Data

In [6]:
df = df.sort_values(by=['creator_id','launched_at'], ascending=True).reset_index(drop = True)
df[['creator_id','id','state','launched_at', 'deadline', 'pledged', 'goal']].head(20)

,creator_id,id,state,launched_at,deadline,pledged,goal
0,3,558744194,successful,2009-09-24 05:33:14,1255546800,3222.00,3000.0
1,3,134175247,successful,2015-06-16 20:02:59,1435726740,11486.00,5000.0
2,4,1658444863,successful,2009-08-28 01:26:16,1251754200,19.00,12.0
3,8,735650730,successful,2011-05-31 14:33:03,1308061983,8554.19,500.0
4,8,1577455391,successful,2014-05-14 22:17:37,1401315457,16167.71,500.0
5,8,171893227,successful,2017-01-04 18:28:57,1484186400,207.66,50.0
6,9,499552311,successful,2009-05-12 18:50:44,1249110000,8647.79,2000.0
7,9,547078236,successful,2012-05-22 18:19:33,1339801200,175511.00,125000.0
8,9,599992665,successful,2014-05-07 16:58:23,1401487200,104983.42,30000.0
9,11,405307432,successful,2009-05-01 15:44:25,1244185140,4100.60,4000.0


# Create Variables

## Create dataframe for 1st project only

In [7]:
# Copy dataframe
base = df.copy()

# First, let's flag the first project for each creator
base['is_first_project'] = base.groupby('creator_id').cumcount() == 0

# Now, we filter out rows where 'is_first_project' is False
base = base[base['is_first_project'] == True].sort_values(by=['creator_id','launched_at'], ascending=True).reset_index(drop = True)

# Set creator id as index
base.set_index('creator_id', inplace=True)

## Create dataframe for 2nd project only

In [8]:
# copy dataframe
next_campaign = df.copy()

# Get all the entrepreneurs' next campaign
next_campaign['is_second_project'] = next_campaign.groupby('creator_id').cumcount() == 1

# Filter out where is_second_project is True
next_campaign = next_campaign[next_campaign['is_second_project'] == True].sort_values(by=['creator_id','launched_at'], ascending=True).reset_index(drop = True)

# Check head
next_campaign[['creator_id','id','state','launched_at', 'deadline', 'pledged', 'goal', 'usd_pledged', 'usd_goal']].head()

,creator_id,id,state,launched_at,deadline,pledged,goal,usd_pledged,usd_goal
0,3,134175247,successful,2015-06-16 20:02:59,1435726740,11486.00,5000.0,11486.000000,5000.0000
1,8,1577455391,successful,2014-05-14 22:17:37,1401315457,16167.71,500.0,16167.710000,500.0000
2,9,547078236,successful,2012-05-22 18:19:33,1339801200,175511.00,125000.0,175511.000000,125000.0000
3,11,1894918555,failed,2009-08-25 03:52:24,1257660060,295.00,2500.0,295.000000,2500.0000
4,32130,1271338092,failed,2024-01-28 11:30:01,1709465401,684.00,2500.0,743.068763,2715.8946


## Variables for Next Campaign

In [9]:
# Create dummy for success, failed, and canceled
next_campaign['success_next'] = np.where(next_campaign['state'] == 'successful', 1, 0)
next_campaign['failed_next'] = np.where(next_campaign['state'] == 'failed', 1, 0)
next_campaign['canceled_next'] = np.where(next_campaign['state'] == 'canceled', 1, 0)

# Create column raised next
next_campaign['raised_next'] = np.where(next_campaign['success_next'] == 1, next_campaign['pledged'], 0)
next_campaign['raised_next_usd'] = np.where(next_campaign['success_next'] == 1, next_campaign['usd_pledged'], 0)


# Map variables to first campaign
next_campaign = next_campaign[['creator_id', 'success_next', 'failed_next', 'canceled_next', 'raised_next', 'raised_next_usd']]
# Set creator id as index
next_campaign.set_index('creator_id', inplace=True)


In [10]:
# Merge with base
base = base.merge(next_campaign, on = 'creator_id', how='left')

In [11]:
base[['id', 'state', 'goal', 'pledged', 'raised_next', 'success_next', 'failed_next', 'canceled_next', 'raised_next_usd']].head()

,id,state,goal,pledged,raised_next,success_next,failed_next,canceled_next,raised_next_usd
creator_id,,,,,,,,,
3,558744194,successful,3000.0,3222.00,11486.00,1.0,0.0,0.0,11486.00
4,1658444863,successful,12.0,19.00,NaN,NaN,NaN,NaN,NaN
8,735650730,successful,500.0,8554.19,16167.71,1.0,0.0,0.0,16167.71
9,499552311,successful,2000.0,8647.79,175511.00,1.0,0.0,0.0,175511.00
11,405307432,successful,4000.0,4100.60,0.00,0.0,1.0,0.0,0.00


## Variables for all campaigns after first

In [12]:
# Copy dataframe
test = df.copy()

# First, let's flag the first project for each creator
test['is_first_project'] = test.groupby('creator_id').cumcount() == 0

# Now, we filter out rows where 'is_first_project' is False
test = test[test['is_first_project'] == False]

# Append the first project launched at colmn by each creator as new column 
test = test.merge(df.groupby('creator_id')['launched_at'].first().reset_index().rename(columns={'launched_at':'first_project_launched_at'}), on='creator_id')

# Reset index if needed
test = test.reset_index(drop=True)

test[['creator_id','id','state', 'state_changed_at','launched_at', 'deadline', 'pledged', 'goal', 'is_first_project', 'first_project_launched_at']].head()

,creator_id,id,state,state_changed_at,launched_at,deadline,pledged,goal,is_first_project,first_project_launched_at
0,3,134175247,successful,2015-07-01 04:59:02,2015-06-16 20:02:59,1435726740,11486.00,5000.0,False,2009-09-24 05:33:14
1,8,1577455391,successful,2014-05-28 22:17:37,2014-05-14 22:17:37,1401315457,16167.71,500.0,False,2011-05-31 14:33:03
2,8,171893227,successful,2017-01-12 02:00:01,2017-01-04 18:28:57,1484186400,207.66,50.0,False,2011-05-31 14:33:03
3,9,547078236,successful,2012-06-15 23:00:18,2012-05-22 18:19:33,1339801200,175511.00,125000.0,False,2009-05-12 18:50:44
4,9,599992665,successful,2014-05-30 22:00:10,2014-05-07 16:58:23,1401487200,104983.42,30000.0,False,2009-05-12 18:50:44


Create key variables

In [13]:
# Calculate total number of Kickstarter campaigns for each creator after first one
test.groupby('creator_id', as_index = False)['id'].count().sort_values(by = 'creator_id', ascending = True).rename(columns = {'id':'total_campaigns_after_first'}).head()

,creator_id,total_campaigns_after_first
0,3,1
1,8,2
2,9,2
3,11,4
4,32130,3


In [14]:
# Calculate total number of successful, failed, canceled Kickstarter campaigns for each creator after first one
successful = dict(test[test['state'] == 'successful'].groupby('creator_id')['id'].count().rename('total_successful_campaigns_after_first'))
failed = dict(test[test['state'] == 'failed'].groupby('creator_id')['id'].count().rename('total_failed_campaigns_after_first'))
canceled = dict(test[test['state'] == 'canceled'].groupby('creator_id')['id'].count().rename('total_canceled_campaigns_after_first'))
total = dict(test.groupby('creator_id')['id'].count().rename('total_campaigns_after_first'))
total_raised = dict(test[test['state'] == 'successful'].groupby('creator_id')['pledged'].sum().rename('total_raised_after_first'))
total_raised_usd = dict(test[test['state'] == 'successful'].groupby('creator_id')['usd_pledged'].sum().rename('total_raised_usd_after_first'))


# Calculate total number of successful, failed, canceled Kickstarter campaigns for each creator after first one for the next 3 years
successful_3 = dict(test[(test['state'] == 'successful') & (test['launched_at'] <= test['first_project_launched_at'] + pd.DateOffset(years=3))].groupby('creator_id')['id'].count().rename('total_successful_campaigns_after_first_3'))
failed_3 = dict(test[(test['state'] == 'failed') & (test['launched_at'] <= test['first_project_launched_at'] + pd.DateOffset(years=3))].groupby('creator_id')['id'].count().rename('total_failed_campaigns_after_first_3'))
canceled_3 = dict(test[(test['state'] == 'canceled') & (test['launched_at'] <= test['first_project_launched_at'] + pd.DateOffset(years=3))].groupby('creator_id')['id'].count().rename('total_canceled_campaigns_after_first_3'))
total_3 = dict(test[test['launched_at'] <= test['first_project_launched_at'] + pd.DateOffset(years=3)].groupby('creator_id')['id'].count().rename('total_campaigns_after_first_3'))
total_raised_3 = dict(test[(test['state'] == 'successful') & (test['launched_at'] <= test['first_project_launched_at'] + pd.DateOffset(years=3))].groupby('creator_id')['pledged'].sum().rename('total_raised_after_first_3'))
total_raised_usd_3 = dict(test[(test['state'] == 'successful') & (test['launched_at'] <= test['first_project_launched_at'] + pd.DateOffset(years=3))].groupby('creator_id')['usd_pledged'].sum().rename('total_raised_usd_after_first_3'))

Create dataset of entrepreneurs' first projects only

In [15]:
# Map outcomes successful, failed, canceled to new dataframe
base['total_successful_after_first'] = base.index.map(successful)
base['total_failed_after_first'] = base.index.map(failed)
base['total_canceled_after_first'] = base.index.map(canceled)
base['total_campaigns_after_first'] = base.index.map(total)
base['total_raised_after_first'] = base.index.map(total_raised)
base['total_raised_after_first_usd'] = base.index.map(total_raised_usd)
# Map outcomes successful, failed, canceled to new dataframe for the next 3 years
base['total_successful_after_first_3'] = base.index.map(successful_3)
base['total_failed_after_first_3'] = base.index.map(failed_3)
base['total_canceled_after_first_3'] = base.index.map(canceled_3)
base['total_campaigns_after_first_3'] = base.index.map(total_3)
base['total_raised_after_first_3'] = base.index.map(total_raised_3)
base['total_raised_after_first_usd_3'] = base.index.map(total_raised_usd_3)


base[['is_first_project', 'total_successful_after_first', 'total_failed_after_first', 'total_canceled_after_first', 
      'total_campaigns_after_first', 'total_raised_after_first', 'total_raised_after_first_usd',
      'total_successful_after_first_3', 'total_failed_after_first_3', 
         'total_canceled_after_first_3', 'total_campaigns_after_first_3', 'total_raised_after_first_3', 'total_raised_after_first_usd_3']].head()

,is_first_project,total_successful_after_first,total_failed_after_first,total_canceled_after_first,total_campaigns_after_first,total_raised_after_first,total_raised_after_first_usd,total_successful_after_first_3,total_failed_after_first_3,total_canceled_after_first_3,total_campaigns_after_first_3,total_raised_after_first_3,total_raised_after_first_usd_3
creator_id,,,,,,,,,,,,,
3,True,1.0,NaN,NaN,1.0,11486.00,11486.00,NaN,NaN,NaN,NaN,NaN,NaN
4,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,True,2.0,NaN,NaN,2.0,16375.37,16375.37,1.0,NaN,NaN,1.0,16167.71,16167.71
9,True,2.0,NaN,NaN,2.0,280494.42,280494.42,NaN,NaN,NaN,NaN,NaN,NaN
11,True,2.0,1.0,1.0,4.0,23409.51,23409.51,NaN,1.0,NaN,1.0,NaN,NaN


In [16]:
# Dump created dataframe to json
json_str = base.to_json(orient='records', lines=True)

with open(r"C:\Users\d0tam\kickstarter-JMP\data\creator_first_project.json", 'w') as f:
    f.write(json_str)